### imprting libraries

In [1]:

# Load the libraries
# special IPython command to prepare the notebook for matplotlib and other libraries
%pylab inline
import pandas as pd
from datetime import datetime
import pyodbc
import xlrd
import time
import math
import decimal
import numpy as np
import html5lib
import numpy as np
#import pandas_profiling
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import sqlite3
# special matplotlib argument for improved plots
from matplotlib import rcParams
sns.set_style("whitegrid")
sns.set_context("poster")
from pandas import DataFrame
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6
from datetime import date


Populating the interactive namespace from numpy and matplotlib


In [3]:
cs = ("DRIVER={ODBC Driver 13 for SQL Server};"
      "SERVER=************;"
      "DATABASE=***********;"
      "Charset=utf-8;"
      "uid=sqladmin;"
      "pwd={*******};"
      )
conn = pyodbc.connect(cs)
cursor = conn.cursor()


### Loading Data

In [5]:
#cursor.execute('select * from ESI_Financials.dbo.Commodity_ESI')
#for row in cursor:
   # print (row)
sql = "select distinct E.*,S.* \
from ESI_Financials.dbo.Commodity_ESI AS E \
LEFT JOIN ESI_Financials.dbo.SSD_HHD_Family S on E.[Dell Part Num] = S.dell_item_nbr \
where E.[Transaction Type] = 'Net Demand'  \
and  E.[Reporting Unit] = 'Configured PE'  \
and E.Global_Parent_Name not like 'JD.com'\
and E.branddesc not like 'POWEREDGE R930'\
and E.transactiondateyear >= 2017\
and E.[Commodity Name] <> 'Chassis'"
data = pd.read_sql(sql,conn)

In [1]:
data.shape

In [2]:
data.head(5)

## Types of Data Preprocessing:

### 1- Data Cleanig
### 2- Data Reduction

## Data Cleaning

### 1- Fill in missing values:
 #####  a- Ignore the Tuple
 #####  b- Use the mean to fill the missing values
 #####  c- Predict missing valuse using varios learing algorithms

In [10]:
# find out the columns that have more than 70 % null
nan_ratio_array = np.round(data.isnull().sum()/data.shape[0])

In [14]:
# write the column names that are more than 70% null
nan_ratio_array[nan_ratio_array >=0.7].index

In [14]:
# drop the columns with more than 70% null values (axis = 1 means column)
L1=data.drop((nan_ratio_array[nan_ratio_array >= 0.7].index),axis=1)

In [16]:
# usually drop columns that have more than 50% null value. in this case, it is fine . no need to drop more

nan_ratio_array_2 = (L1.isnull().sum()/data.shape[0])
nan_ratio_array_2.max()

In [15]:
#Remove columns that are "NUM"
IDcol = []
for col in L1.columns:
   if "ID" in col.upper():
      IDcol.append(col)
IDcol

In [17]:
L1.shape

In [18]:
# select columns that theeir type is not object AND number of unique >15
L1=L1.loc[:,~((L1.dtypes == object) & (L1.nunique() == 1))]
L1.shape


In [ ]:
L1=L1.fillna(L1.mean()) 

In [29]:
import seaborn
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

In [31]:
data["Part Quantity Extended Old"] = data["Part Quantity Extended"]

In [32]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()

In [33]:
#Scale data between 0 and 1
def scaleColumns(df, cols_to_scale):
    for col in cols_to_scale:
        df[col] = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data[col])),columns=[col])
    return df

In [34]:
scaled_df = scaleColumns(data,['Part Quantity Extended'])

In [3]:
data[["Part Quantity Extended","Part Quantity Extended Old"]].head(5)

In [4]:
fig = plt.figure(figsize=(1,1))
df_items = data[["TransactionDateMonth","Part Quantity Extended Old"]]
df_items.groupby("TransactionDateMonth").sum().plot(kind="bar")
plt.savefig('C:/Users/laya_pouresmaeil/Desktop/figure.pdf') 

In [5]:
df_items = data[["TransactionDateMonth","Part Quantity Extended"]]
df_items.groupby("TransactionDateMonth").sum().plot(kind="bar")
#plt.savefig('C:/Users/laya_pouresmaeil/Desktop/figure2.pdf') 

In [38]:
data_Americas = data[data["RegionDesc"]== 'Americas'] 

In [6]:
df_items = data_Americas[["TransactionDateMonth","Part Quantity Extended"]]
df_items.groupby("TransactionDateMonth").sum().plot(kind="bar")
#plt.savefig('C:/Users/laya_pouresmaeil/Desktop/figure2.pdf') 

In [7]:
df_items = data[["TransactionDateMonth","Part Quantity Extended"]]
a=df_items.groupby("TransactionDateMonth").sum()['Part Quantity Extended'].reset_index().sort_values("Part Quantity Extended", ascending=False).head(10)
#a.groupby('TransactionDateMonth').sum()['Part Quantity Extended'].plot(kind="bar")


In [43]:
column = ["ESIUnique_PartNumber","BaseSystemCode","CPU Gen","CPU Vendor","Capacity","Dell Segment Name","Dim","FamilyDesc","FamilyName","Form Factor","Global_Parent_Name","IDC Mainstream DOHS","IDC Product","Interface","ProductDesc","RegionDesc","SegDesc","Sockets","Speed","Supplier","Swimlane","branddesc","country","intel vertical","product generation","sales owner","sub group","vertical parent"]

In [8]:
for col in range(len(column)):
    df_items = data[["TransactionDateMonth", column[col],"Part Quantity Extended"]]
    #a=df_items.groupby(['TransactionDateMonth',column[col]]).sum()['Part Quantity Extended'].reset_index().sort_values("Part Quantity Extended", ascending=False).head(20)
    df_items.groupby(['TransactionDateMonth',column[col]]).sum()['Part Quantity Extended'].unstack().plot.bar(stacked=True)
    plt.savefig('C:/Users/laya_pouresmaeil/Desktop/Forecasting Model/Graph/Server/BarChart/'+column[col]+'.png')               

In [9]:
# Check 2018M03 for now

data_2018M03 = data[data["TransactionDateMonth"]== '2018M03']
data_2018M03

In [10]:

# How many entries are there for Customer?
data_2018M03['Global_Parent_Name'].value_counts().head(5)

In [11]:
data.std()

In [12]:
for col in range(len(column)):
    df_items = data[["TransactionMonthNew", column[col],"Part Quantity Extended"]]
    #a=df_items.groupby(['TransactionMonthNew',column[col]]).sum()['Part Quantity Extended'].reset_index().sort_values("Part Quantity Extended", ascending=False).head(10)
    df_items.groupby(['TransactionMonthNew',column[col]]).sum()['Part Quantity Extended'].unstack().plot(kind="line")
    plt.savefig('C:/Users/laya_pouresmaeil/Desktop/Forecasting Model/Graph/Server/LineChart_Month/'+column[col]+'.png')               

In [67]:
data_Part_Valume=data.groupby(['Dell Part Num']).sum()['Part Quantity Extended Old'].reset_index().sort_values("Part Quantity Extended Old", ascending=False)

In [13]:
data.describe()

In [ ]:
data[["Part Quantity Extended Old"]].sort_values("Part Quantity Extended Old", ascending=False).plot(kind="bar")